In [21]:
import numpy as np
import re
import json
import torch.utils.data
from torch.utils.data import DataLoader
import io,os
from torch import nn
from gensim.models import word2vec
from torch.nn.utils.rnn import pad_sequence
from torch import optim
from torch.autograd import Variable
from torchtext import data

BATCH_SIZE = 100
device = torch.device("cuda")

tag_to_ix = {'start_tag':0,'stop_tag':29,'pad_tag':30}

working_path = '/home/jongsu/jupyter/pytorch_dialogue_ie/'
WV_PATH = '/home/jongsu/jupyter/pytorch_dialogue_ie/parameter/dialogue_wv'


In [22]:
hidden_state_size = 100
batch_size = 100

In [24]:
class encoder(nn.Module):
    def __init__(self, hidden_state_size, batch_size, is_tag_, tag_size, bidir):
        super(encoder, self).__init__()
        self.h_size = hidden_state_size
        self.is_tag = is_tag_
        self.t_size = tag_size
        self.bid = bidir
        self.gru = nn.GRU(self.h_size, self.h_size, bidirectional = self.bid)
        self.embed_tag = nn.Embedding(self.tag_size, self.h_size)
        self.combinput = nn.Linear(2*self.h_size, self.h_size)
        
    def forward(self, input_, input_tag, hidden_state):
        if self.is_tag == False:
            output, hidden_state = self.gru(input_, hidden_state)
        else:
            emb_tag = self.embed_tag(input_tag)
            input_ = self.combinput(torch.cat(emb_tag, input_))
            output, hidden_state = self.gru(input_, hidden_state)
        return output, hidden_state
    
    def init_hidden():
        hidden = torch.zeros(100,100)
        return hidden
    
    
    
class decoder(nn.Module): #target padding sos
    def __init__(self, hidden_size, batch_size):
        super(decoder, self).__init__()
        self.gru = nn.GRU(hidden_state_size, hidden_state_size)
        
    def forward(self, input_, hidden_state):
        output, hidden_state = self.gru(input_, hidden_state)
        return output, hidden_state
    
     def init_hidden():
        hidden = torch.zeros(100,100)
        return hidden

In [25]:
encoder1 = encoder(hidden_state_size,
                   batch_size,
                   is_tag_ = False,
                   tag_size = 31, 
                   bidir = True
                  )

encoder2 = encoder(hidden_state_size,
                   batch_size,
                   is_tag_ = True,
                   tag_size = 31, 
                   bidir = True
                  )

decoder1 = decoder(hidden_state_size,
                   batch_size
                  )

In [ ]:

    
for batch_data in batchload(train_data, repeat=True, batchsize = batch_size, data_seq = dataseq ):
    #load txt data from jsonfile

    shared_model.zero_grad()
    comp_model.zero_grad()

    new_dial, new_tag, dial_leng = all_preprocess(shared_model, batch_data) #we need split batch_data
    #load batch*(dialogue_length*sent_vec(float)) -> new_dial
    #load batch*tag -> new_tag
    #load batch * dial_leng

    loss = comp_model.neg_log_likelihood(make_mask(dial_leng), new_dial, new_tag, BATCH_SIZE)
    loss,newary_ = loss_filtering(loss,filtering_value, newary_,k)
    batch_loss = torch.sum(loss)
    batch_loss.backward(retain_graph=False)
    optimizer1.step()
    optimizer2.step()

    unuselist = [new_dial, new_tag, dial_leng]
    del unuselist

In [18]:
def train_func(train_data, shared_model, comp_model, dataseq, filtering_value, iter_num, batch_size, learning_rate):
    """
    Args:
            train_data: 
                train data
                
            shared_model:
                shared model
                
            comp_model:
                comp model
                
            dataseq:
                data sort sequence
                
            filtering_value:
                allowed maximum loss
                
            iter_num:
                train iterate
                
            batch_size:
                batchsize
            
            learning_rate:
                learning_rate

    Yields:
            newary:
                not trained data
                which has loss bigger then filtering value
    
    """
    
    optimizer1 = optim.SGD(shared_model.parameters(), lr= learning_rate, weight_decay=1e-4)
    optimizer2 = optim.SGD(comp_model.parameters(), lr= learning_rate, weight_decay=1e-4)
    
    newary_ = []
    k = 0
    for batch_data in batchload(train_data, repeat=True, batchsize = batch_size, data_seq = dataseq ):
        #load txt data from jsonfile

        shared_model.zero_grad()
        comp_model.zero_grad()
        
        new_dial, new_tag, dial_leng = all_preprocess(shared_model, batch_data) #we need split batch_data
        #load batch*(dialogue_length*sent_vec(float)) -> new_dial
        #load batch*tag -> new_tag
        #load batch * dial_leng

        loss = comp_model.neg_log_likelihood(make_mask(dial_leng), new_dial, new_tag, BATCH_SIZE)
        loss,newary_ = loss_filtering(loss,filtering_value, newary_,k)
        batch_loss = torch.sum(loss)
        batch_loss.backward(retain_graph=False)
        optimizer1.step()
        optimizer2.step()

        unuselist = [new_dial, new_tag, dial_leng]
        del unuselist
        
        k = k + 1
        
        if k == 1:
            break
            
        print(k)
        if k%10 != 0:
            torch.save(shared_model.state_dict(),working_path + 'parameter/shared.pth')
            torch.save(comp_model.state_dict(),working_path + 'parameter/crf_gru.pth') #3.53 save with dummy
            dummy_input = [make_mask(dial_leng),new_dial]
            
            print("tag = ",new_tag[7])
            print("expect = ",comp_model(BATCH_SIZE,dummy_input,seq=7)[1])
            print("accuracy = ", cal_accuracy(comp_model(BATCH_SIZE,dummy_input,seq=7)[1],new_tag[7]))
            
            print(loss)
        if k == int(len(train_data)/BATCH_SIZE)*iter_num:
            break
            
        if k % int(len(train_data)/BATCH_SIZE) == 0:
            newary = newary_
            newary_ = []
    print(newary)
    
    return newary

tensor([[[-6.4010e-02,  7.1903e-01,  6.1823e-01,  ...,  3.9984e-01,
          -1.0932e-01, -1.7312e+00],
         [-1.9888e+00,  3.0375e-01, -1.2334e+00,  ...,  1.1662e+00,
          -4.0237e-01, -5.0372e-01],
         [-6.3268e-01,  1.6851e+00, -7.6740e-01,  ..., -1.2149e+00,
          -2.0391e-01,  4.4209e-01],
         ...,
         [ 7.6604e-01, -4.5329e-01, -9.5204e-01,  ..., -1.1796e-01,
          -7.3638e-01,  1.3736e+00],
         [-2.7988e-01, -1.0790e-01, -1.1011e+00,  ..., -1.1013e-01,
           1.9102e-01,  1.6579e+00],
         [-1.5484e+00,  6.2958e-01, -8.7390e-01,  ...,  1.4543e+00,
          -5.9910e-01, -2.7030e-01]],

        [[-3.3939e-01, -6.0895e-01,  1.0888e+00,  ..., -3.5105e-01,
           5.6317e-01,  7.8285e-01],
         [ 4.8802e-01, -8.6820e-02, -1.8796e+00,  ..., -4.4382e-01,
           2.1503e+00, -1.5849e+00],
         [ 8.2981e-02,  1.4814e+00,  5.5677e-01,  ..., -9.6178e-01,
           1.7989e+00, -2.1169e+00],
         ...,
         [-3.2553e-01,  9

(tensor([[[-1.1793e-01,  1.1792e+00, -3.3370e-01,  ..., -4.9448e-02,
           -6.8771e-01,  2.3439e-01],
          [ 1.2098e-01,  4.0973e-02, -6.6107e-01,  ...,  3.8152e-01,
           -2.4340e-01, -2.4051e-01],
          [-1.0198e+00, -5.5639e-01,  8.2766e-02,  ...,  7.6325e-01,
           -5.8140e-01, -2.4245e-01],
          ...,
          [ 7.6781e-01,  9.8579e-01, -3.4716e-01,  ..., -6.5801e-03,
           -2.9974e-01, -1.8975e-02],
          [ 5.0546e-01,  8.6346e-01,  3.1279e-02,  ...,  5.3067e-01,
            4.0330e-01,  2.0547e-01],
          [-6.1136e-01, -5.2947e-01,  9.5464e-01,  ..., -4.0955e-01,
            4.8972e-01,  2.4979e-01]],
 
         [[ 6.8290e-02,  5.9337e-01,  1.4649e-01,  ...,  1.7725e-02,
           -2.6069e-01,  1.1376e-01],
          [-4.1837e-01,  1.7344e-01, -7.1782e-01,  ...,  3.5297e-01,
            1.0536e-01, -5.4529e-01],
          [-8.5074e-01, -1.2458e-01, -3.8070e-01,  ...,  7.8022e-01,
           -1.1806e+00, -1.2204e-01],
          ...,
    

tensor([[ 0.9719,  0.1587,  0.8667,  0.4308,  0.3225,  0.7839,  0.3232,
          0.3374],
        [ 0.8198,  0.9398,  0.3241,  0.7783,  0.0813,  0.9192,  0.3575,
          0.4030],
        [ 0.9136,  0.0059,  0.7340,  0.9296,  0.5289,  0.7905,  0.6123,
          0.6578]], dtype=torch.float64)